<a href="https://colab.research.google.com/github/Belouadif/marianmt-darija-translation/blob/main/l1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install sacrebleu

import transformers
import datasets
print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.1 MB/s eta 0:00:00
Tra

In [ ]:
import pandas as pd
from google.colab import files

print("Please upload your dataset (CSV file containing 'Darija', 'English', and optionally 'Darija ar' columns).")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name, delimiter=';', on_bad_lines='skip')

required_columns = ['darija', 'eng']
if all(col in data.columns for col in required_columns):
    data = data[required_columns]
    print("Dataset loaded and filtered successfully! Displaying the first few rows:")
    print(data.head())
else:
    print(f"Error: The dataset must contain the following columns: {required_columns}")

Please upload your dataset (CSV file containing 'Darija', 'English', and optionally 'Darija ar' columns).


Saving darija_english.csv to darija_english (2).csv
Dataset loaded and filtered successfully! Displaying the first few rows:
                                   darija  \
0    homa mkhbbyin chi haja, ana mti99en!   
1      bayna homa tay7awlo ib9aw mbrrdin.   
2  loTilat mabaynach fihom mori7in bzzaf.   
3      ghaliban ghayjrriw 3lih mn lkhdma!   
4                     Tab3an rah mkta2eb!   

                                                 eng  
0                They're hiding something, I'm sure!  
1    It's obvious they're trying to keep their cool.  
2            the hotels don't seem very comfortable.  
3  he is probably about to be laid off by head of...  
4                         of course he's depressive!  


In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 33.3 MB/s eta 0:00:00


In [ ]:
data["darija"] = data["darija"].astype(str)
data["eng"] = data["eng"].astype(str)

def tokenize_function(examples):
    inputs = tokenizer(examples[source_lang], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
    targets = tokenizer(examples[target_lang], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
    return inputs, targets

tokenized_inputs = []
tokenized_targets = []

for i in range(len(data)):
    inputs, targets = tokenize_function({
        source_lang: data.iloc[i]["darija"],
        target_lang: data.iloc[i]["eng"]
    })
    tokenized_inputs.append(inputs)
    tokenized_targets.append(targets)

print("Tokenization complete!")


Tokenization complete!


In [ ]:
import torch
from torch.utils.data import Dataset

class TranslationDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx],
        }

train_dataset = TranslationDataset(
    {"input_ids": torch.cat([torch.tensor(x["input_ids"]) for x in tokenized_inputs]),
     "attention_mask": torch.cat([torch.tensor(x["attention_mask"]) for x in tokenized_inputs])},
    {"input_ids": torch.cat([torch.tensor(x["input_ids"]) for x in tokenized_targets])}
)

print(f"Training dataset size: {len(train_dataset)}")

<ipython-input-11-93607c1e8ae2>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  {"input_ids": torch.cat([torch.tensor(x["input_ids"]) for x in tokenized_inputs]),
<ipython-input-11-93607c1e8ae2>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.cat([torch.tensor(x["attention_mask"]) for x in tokenized_inputs])},
<ipython-input-11-93607c1e8ae2>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  {"input_ids": torch.cat([torch.tensor(x["input_ids"]) for x in tokenized_targets])}


Training dataset size: 45378


In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import MarianMTModel, MarianTokenizer, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
train_data = [
    {"darija": "homa mkhbbyin chi haja", "eng": "They're hiding something"},
    {"darija": "bayna homa tay7awlo ib9aw mbrrdin", "eng": "It's obvious they're trying to keep their cool"}
]

eval_data = [
    {"darija": "loTilat mabaynach fihom mori7in bzzaf", "eng": "The hotels don't seem very comfortable"}
]

In [ ]:
train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"
tokenizer = MarianTokenizer.from_pretrained(model_name)

def preprocess_function(example):
    inputs = tokenizer(example["darija"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(example["eng"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = train_dataset.map(preprocess_function)
eval_dataset = eval_dataset.map(preprocess_function)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    num_train_epochs=3,              # Number of epochs
    per_device_train_batch_size=16,  # Batch size
    save_steps=10_000,               # Save checkpoint every 10k steps
    save_total_limit=2,              # Keep only the last 2 checkpoints
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=5e-5,              # Learning rate
    logging_dir="./logs",            # Directory for logs
    logging_steps=500,               # Log every 500 steps
)

In [ ]:
model = MarianMTModel.from_pretrained(model_name)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,1.868999
2,No log,0.800035
3,No log,0.734511


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3, training_loss=2.8058671951293945, metrics={'train_runtime': 57.9554, 'train_samples_per_second': 0.104, 'train_steps_per_second': 0.052, 'total_flos': 203390189568.0, 'train_loss': 2.8058671951293945, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/source.spm',
 './trained_model/target.spm',
 './trained_model/added_tokens.json')

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00


In [ ]:
import evaluate

bleu = evaluate.load("bleu")

test_sentences = [
    {"darija": "homa mkhbbyin chi haja", "eng": "They're hiding something."},
    {"darija": "ana mti99en!", "eng": "I'm sure!"},
]

predictions = []
references = []

for sample in test_sentences:
    inputs = tokenizer(sample["darija"], return_tensors="pt", max_length=128, truncation=True)
    translated = model.generate(**inputs)
    prediction = tokenizer.decode(translated[0], skip_special_tokens=True)

    predictions.append(prediction)
    references.append([sample["eng"]])

result = bleu.compute(predictions=predictions, references=references)
print("BLEU Score:", result["bleu"])

BLEU Score: 0.0
